## **Import important Libraries**

In [10]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

### **Softmax in numpy**

In [2]:
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x), axis=0)

x = np.array([2.0, 1.0, 0.1])
outputs = softmax(x)
print('softmax numpy: ', outputs)

softmax numpy:  [0.65900114 0.24243297 0.09856589]


### **Softmax in pytorch**

In [3]:
x = torch.tensor([2.0, 1.0, 0.1])
outputs = torch.softmax(x, dim=0)
print('softmax pytorch: ', outputs)

softmax pytorch:  tensor([0.6590, 0.2424, 0.0986])


### **Cross Entropy Loss in numpy**

In [9]:
def cross_entropy(actual, predicted):
    loss = -np.sum(actual * np.log(predicted))
    return loss # / float(predicted.shape[0])

# y must be one hot encoded
# if class 0: [1 0 0]
# if class 1: [0 1 0]
# if class 2: [0 0 1]
Y = np.array([1, 0, 0])

# y_pred has probabilities
Y_pred_good = np.array([0.7, 0.2, 0.1])
Y_pred_bad = np.array([0.1, 0.3, 0.6])

l1 = cross_entropy(Y, Y_pred_good)
l2 = cross_entropy(Y, Y_pred_bad)

print(f'Loss1 numpy: {l1:.4f}')
print(f'Loss2 numpy: {l2:.4f}')

Loss1 numpy: 0.3567
Loss2 numpy: 2.3026


### **Cross Entropy Loss in pytorch**

In [16]:
loss = nn.CrossEntropyLoss()

# correct class label, not one encoded
Y = torch.tensor([0])
# size of Y prediction => nsamples * nclasses = 1*3
# and this is raw value and can't apply softmax
Y_pred_good = torch.tensor([[2.0, 1.0, 0.1]])
Y_pred_bad = torch.tensor([[0.5, 2.0, 0.3]])

l1 = loss(Y_pred_good, Y)
l2 = loss(Y_pred_bad, Y)

print(f'Loss1 pytorch: {l1.item():.4f}')
print(f'Loss2 pytorch: {l2.item():.4f}')

# Actual value of prediction
_, predictions1 = torch.max(Y_pred_good, 1)
_, predictions2 = torch.max(Y_pred_bad, 1)

print(f'Prediction1 pytorch: {predictions1}')
print(f'Prediction2 pytorch: {predictions2}')

Loss1 pytorch: 0.4170
Loss2 pytorch: 1.8406
Prediction1 pytorch: tensor([0])
Prediction2 pytorch: tensor([1])


#### **Multiple Samples**

In [17]:
loss = nn.CrossEntropyLoss()

# correct class label, not one encoded
# 3 samples
Y = torch.tensor([2, 0, 1])
# size of Y prediction => nsamples * nclasses = 3*3
# and this is raw value and can't apply softmax
Y_pred_good = torch.tensor([[0.1, 1.0, 2.1], [2.0, 1.0, 0.1], [0.1, 3.0, 0.1]])
Y_pred_bad = torch.tensor([[2.1, 1.0, 0.1], [0.1, 1.0, 2.1], [0.1, 3.0, 0.1]])

l1 = loss(Y_pred_good, Y)
l2 = loss(Y_pred_bad, Y)

print(f'Loss1 pytorch: {l1.item():.4f}')
print(f'Loss2 pytorch: {l2.item():.4f}')

# Actual value of prediction
_, predictions1 = torch.max(Y_pred_good, 1)
_, predictions2 = torch.max(Y_pred_bad, 1)

print(f'Prediction1 pytorch: {predictions1}')
print(f'Prediction2 pytorch: {predictions2}')

Loss1 pytorch: 0.3018
Loss2 pytorch: 1.6242
Prediction1 pytorch: tensor([2, 0, 1])
Prediction2 pytorch: tensor([0, 2, 1])


## **Multi Class Classification Neural Network**
#### Using Cross Entropy and not use softmax in last layer

### 1) Cross Entropy Loss

In [18]:
# Multiclass problem
class NeuralNetClassification(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNetClassification, self).__init__()
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        out = self.linear1(x)
        out = self.relu(out)
        out = self.linear2(out)
        # no softmax at the end
        return out
    
model = NeuralNetClassification(input_size=28*28, hidden_size=5, num_classes=3)
criterion = nn.CrossEntropyLoss() # (applies Softmax)

## **Binary Classification Neural Network**
#### Using Binary classification and Use Sigmoid at the end

### 2) Binary Cross Entropy Loss

In [19]:
# Binary class problem
class NeuralNetBinaryClassification(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(NeuralNetBinaryClassification, self).__init__()
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(hidden_size, 1)
    
    def forward(self, x):
        out = self.linear1(x)
        out = self.relu(out)
        out = self.linear2(out)
        # sigmoid at the end
        y_pred = torch.sigmoid(out)
        return out
    
model = NeuralNetBinaryClassification(input_size=28*28, hidden_size=5)
criterion = nn.BCELoss()